In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
throws_df = pd.read_csv('../data/raw/all_games_0704.csv')
non_bricks = throws_df[(throws_df['thrower_x'] != 0) & (throws_df['thrower_y'] != 40)]
non_bricks_clean = non_bricks.dropna(subset=['thrower_x', 'thrower_y'])

/var/folders/r4/cqt7s71n4k18qp94swd6_njr0000gn/T/ipykernel_33504/1692646384.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  throws_df = pd.read_csv('../data/raw/all_games_0704.csv')


In [124]:
def get_bin_data(df, nbinsx, nbinsy):
    hist, xedges, yedges = np.histogram2d(df['thrower_x'], df['thrower_y'], bins=[nbinsx, nbinsy])
    x_coords = []
    y_coords = []
    counts = []
    for i in range(nbinsx):
        for j in range(nbinsy):
            x = (xedges[i] + xedges[i+1]) / 2
            y = (yedges[j] + yedges[j+1]) / 2
            count = hist[i][j]
            x_coords.append(x)
            y_coords.append(y)
            counts.append(count)
    return x_coords, y_coords, counts


def shot_plot(teamID, nbinsx=10, nbinsy=15):
    shots = throws_df[(throws_df.gameID.apply(lambda x:x[:4]) == '2023') & (throws_df['thrower_x'] != 0) & (throws_df['thrower_y'] != 40) & ((throws_df['home_teamID'] == teamID) | (throws_df['away_teamID'] == teamID))].dropna(subset=['thrower_x', 'thrower_y'])
    x_coords, y_coords, counts = get_bin_data(shots, nbinsx, nbinsy)
    # Create the figure
    fig = make_subplots(rows=2, cols=2,
                    row_heights=[0.2, 0.8],
                    column_widths=[0.7, 0.3],
                    vertical_spacing = 0.02,
                    horizontal_spacing = 0.02,
                    shared_yaxes=True,
                    shared_xaxes=False)

    # Add the bar chart
    fig.add_trace(go.Violin(
        x=shots.thrower_x,
        name='',
        hoverinfo='none',
        line_color='#26828E'
    ), row=1, col=1)

    fig.add_trace(go.Violin(
        y=shots.thrower_y,
        name='',
        hoverinfo='none',
        line_color='#26828E'
    ), row=2, col=2)

    # Add the main scatter plot
    fig.add_trace(go.Scatter(
        x=x_coords, y=y_coords, mode='markers', name='markers',
        marker=dict(
            size=counts, sizemode='area', sizeref=(2. * max(counts) / ((300 / nbinsx) ** 2)), sizemin=2.5,
            color=counts,  # Set the color to the counts using the "viridis" color scheme
            colorscale='viridis',  # Set the color scale to "viridis"
            line=dict(width=1, color='#333333'), symbol='circle',
            colorbar=dict(title='Counts')  # Add a color bar with the title "Counts"
        ),
        hovertemplate='Count: %{text}<extra></extra>',
        text=[int(x) for x in counts]
    ), row = 2, col = 1)

    # Add black horizontal lines at y = 20 and y = 100
    fig.add_shape(type='line', x0=-27, y0=20, x1=27, y1=20, line=dict(color='black', width=1), row=2, col=1)
    fig.add_shape(type='line', x0=-27, y0=100, x1=27, y1=100, line=dict(color='black', width=1), row=2, col=1)
    fig.add_shape(type='line', x0=-27, y0=0, x1=27, y1=0, line=dict(color='black', width=1), row=2, col=1)
    fig.add_shape(type='line', x0=-27, y0=120, x1=27, y1=120, line=dict(color='black', width=1), row=2, col=1)
    fig.add_shape(type='line', x0=-27, y0=0, x1=-27, y1=120, line=dict(color='black', width=1), row=2, col=1)
    fig.add_shape(type='line', x0=27, y0=0, x1=27, y1=120, line=dict(color='black', width=1), row=2, col=1)

    fig.update_layout(
        xaxis=dict(range=[-27, 27], showticklabels=False),  # Apply x-axis range to the scatter plot
        yaxis=dict(range=[0, None]),  # Apply y-axis range to the scatter plot
        xaxis2=dict(range=[-27, 27]),
        yaxis2=dict(range=[0, 120]),  
        showlegend=False,
        width=600,
        height=600,
        margin=dict(t=50, b=50, l=50, r=50),
    )
    fig.update_xaxes(showgrid=False, zeroline=False)
    fig.update_yaxes(showgrid=False, zeroline=False)

    # Add image
    fig.layout.images = [dict(
        source="https://theaudl.com/sites/default/files/logo-team-MAD.png",
        xref="paper", yref="paper",
        x=0.1, y=1.05,
        sizex=0.8, sizey=0.8,
        xanchor="center", yanchor="top"
      )]

    # Show the plot
    fig.show()

In [125]:
shot_plot('shred', 10, 15)

In [112]:
shot_plot('phoenix', 10, 15)

In [7]:
shot_plot('mechanix', 15, 20)

In [167]:
shot_plot('empire', 15, 20)

In [168]:
shot_plot('shred', 15, 20)

In [156]:
shot_plot('breeze', 15, 20)

In [147]:
import plotly.graph_objects as go
from PIL import Image

pyLogo = Image.open("../logos/union.png")

# Create the figure
fig = go.Figure()

# Add the main scatter plot
fig.add_trace(go.Scatter(
    x=x_coords, y=y_coords, mode='markers', name='markers',
    marker=dict(
        size=counts, sizemode='area', sizeref=5, sizemin=2.5,
        color=counts,
        line=dict(width=1, color='#333333'), symbol='circle',
    ),
))

# Set the layout properties
fig.update_layout(
    xaxis=dict(range=[-27, 27]),
    yaxis=dict(range=[0, 120]),
    showlegend=False,
    width=600,
    height=600,
    margin=dict(t=50, b=50, l=50, r=50),
)

fig.layout.images = [dict(
        source=pyLogo,
        xref="x", yref="y",
        x=0, y=1,
        sizex=20, sizey=20,
        xanchor="left", yanchor="top"
      )]

# Show the plot
fig.show()


In [127]:
import plotly.plotly as py
import plotly.graph_objs as go

fig = py.get_figure('https://plotly.com/~Dreamshot/8152/', raw=True)
fig['layout']['yaxis']['tickangle'] = 0
fig = go.Figure(fig)

fig.layout.images = [dict(
        source="https://raw.githubusercontent.com/cldougl/plot_images/add_r_img/accuweather.jpeg",
        xref="paper", yref="paper",
        x=0.1, y=1.05,
        sizex=0.4, sizey=0.4,
        xanchor="center", yanchor="bottom"
      )]

py.iplot(fig, fileopt='overwrite', filename='Logos/Florida_Rainfall_AccuWeather')

ImportError: 
The plotly.plotly module is deprecated,
please install the chart-studio package and use the
chart_studio.plotly module instead. 
